## Lab 07

### Zadanie 1 - Regexy

Przejdź 15 zadań ze strony http://regexone.com/

### Zadanie 2 - Regexy

_Atoms_

Napisz program, który (używając wyrażeń regularnych) przefiltruje plik atoms.log tak by pozostały jedynie linie dot. bezproblemowego przebiegu eksperymentu.
czyli np. powinny zostać odrzucone takie linie jak
```
RUN 000039 COMPLETED. OUTPUT IN FILE yttrium.dat. 1 UNDERFLOW WARNING.
```
czy
```
RUN 000058 COMPLETED. OUTPUT IN FILE cerium.dat. ALGORITHM DID NOT CONVERGE AFTER 100000 ITERATIONS.
```
można jedynie założyć, że każda linijka pasuje do poniższego schematu
```
RUN [[NUMER Z DOKŁADNIE 6 CYFR]] COMPLETED. OUTPUT IN FILE [[NAZWA]].dat. [[COKOLWIEK]]
```

In [2]:
import re

pattern = re.compile('RUN \d{6} COMPLETED. OUTPUT IN FILE .+?.dat.$')

with open(r'e:\github\python_tutorial\lab7_files\atoms.txt') as atoms:
    for line in atoms:
        if re.match(pattern, line):
            print(line)
        #if ...:
        #    print(line)

FileNotFoundError: [Errno 2] No such file or directory: 'e:\\github\\python_tutorial\\lab7_files\\atoms.txt'

### Zadanie 3 - Regexy

_Messages_

Napisz program który wypisze linie z logu systemowego "messages.txt" które zawierają informacje o nieprawidłowej nazwie użytkownika (invalid user)

In [3]:
pattern = re.compile("(\w{3}) (\d{2}) (\d{2}:\d{2}):\d{2} .* user (.*) from (\d{3}\.\d{3}\.\d{3}\.\d{3})")

def translate(text):
    return 'Niepoprawna nazwa uzytkownika: ' + text.group(4) + '", polaczenie z ' + text.group(5)

with open(r'e:\github\python_tutorial\lab7_files\messages.txt') as messages:
    for line in messages:
        m = re.search(pattern, line)
        if m:
            print(translate(m))

FileNotFoundError: [Errno 2] No such file or directory: 'e:\\github\\python_tutorial\\lab7_files\\messages.txt'

Przerób program tak, żeby wyświetlał sformatowane komunikaty, tzn.
```
Jun 29 20:18:40 noether sshd[5805]: Invalid user tester from 218.189.194.200
```
powinno zostać przerobione na
```
Niepoprawna nazwa użytkownika: "tester", połaczenie z 218.189.194.200 nawiązano 29 czerwca o godz. 20:18
```

### Zadanie 4 - Collections
Uzupenij ponizsze funckje korzystajac z zaimporotwanych klas

In [4]:
from collections import defaultdict

def count_letters(word):
    """ Zwraca slownik gdzie kluczami sa litery, a wartosci ilosc ich wystepowania """
    output = defaultdict(int)
    for letter in word:
        output[letter] += 1
        
    return output


print(count_letters('aaaaa') == {'a': 5})
print(count_letters('abbccaaaa') == {'a': 5, 'b': 2, 'c': 2})
print(count_letters('abc') == {'a': 1, 'b': 1, 'c': 1})

True
True
True


In [12]:
def group_words_by_first_letter(text):
    """ Dzieli tekst po spacjach i zwraca slownik gdzie kluczami sa pierwsze litery, a wartosciami listy slow zaczynajacych sie na te litery"""
    output = defaultdict(list)
    for splitted in text.split():
        output[splitted[0]].append(splitted)
    return output

print(group_words_by_first_letter("ala ma kota") == {'a': ['ala'], 'm': ['ma'], 'k': ['kota']})
print(group_words_by_first_letter("ala ma kota ala ma psa") == {'a': ['ala', 'ala'], 'm': ['ma', 'ma'], 'k': ['kota'], 'p': ['psa']})   
print(group_words_by_first_letter("ala ma kota ale marysia ma konia") == {'a': ['ala', 'ale'], 'm': ['ma', 'marysia', 'ma'], 'k': ['kota', 'konia']})

True
True
True


### Zadanie 5 - Klasy
Uzupełnij brakujące metody poniższej klasy.

In [7]:
class FrozenDictionary(object):
    """
    Odpowiednik frozenset dla zbiorów, czyli słownik, który nie jest modyfikowalny,
    a dzięki temu może być np. elementem zbiorów, albo kluczem w innym słowniku.
    """
    
    def __init__(self, dictionary):
        """Tworzy nowy zamrożony słownik z podanego słownika"""
        self.dict = dictionary.copy()
    
    def __hash__(self):
        """Zwraca hasz słownika (int)"""
        return hash(frozenset(self.dict.items()))    
    
    def __eq__(self, d):
        """Porównuje nasz słownik z zamrożonym słownikiem d"""
        return self.dict == d
    
    def __repr__(self):
        """Zwraca reprezentację naszego słownika jako string"""
        return self.dict.__repr__()
        
dicts = [FrozenDictionary({'ala': 4}), 
         FrozenDictionary({'ala': 1, 'jacek': 0}),
         FrozenDictionary({'ala': 4}), 
         FrozenDictionary({'ala': 2}), 
         FrozenDictionary({'jacek': 0, 'ala': 1})]

s = set(dicts)
print(dicts[0] == dicts[2])
print(dicts[0] != dicts[3])
print(len(s) == 3)
for d in dicts:
    print(d in s)

# Powinno wyświetlić coś w stylu set([{'ala': 4}, {'ala': 1, 'jacek': 0}, {'ala': 2}])
print(s)

True
True
True
True
True
True
True
True
{{'ala': 1, 'jacek': 0}, {'ala': 4}, {'ala': 2}}


### Zadanie 6 - Bag of Words
Napisz klasę "Bag of words", która będzie "workiem słów" z zadanego dokumentu. Oznacza to, że ma w sobie przechowywać informacje o tym ile razy każde słowo wystąpiło w dokumencie. Dodatkowo ma udostępniać następującą funkcjonalność:
* Można utworzyć go na dwa sposoby:
```
bow = BagOfWords("ala ma kota ala ma ala")
bow = BagOfWords(open("plik.txt"))
```
* Możliwość wyświetlania go po prostu pisząc print(bow) (kolejność taka sama jak przy przeglądaniu forem)
```
ala:3, ma:2, kota:1
```
* Sprawdzanie czy jakieś słowo należy do naszego worka pisząc
```
if 'ala' in bow:
    ...
```
* Przeglądanie słów w worku od najczęściej do najrzadziej występującego, czyli
```
for word in bow:
  print(word)
```
    powinniśmy zobaczyć
```
ala
ma
kota
```
* Możliwość dodawania dwóch worków, pisząc
```
bow1 = BagOfWords("ala ma kota ala ma ala")
bow2 = BagOfWords("tomek tez ma kota")
bow3 = bow1 + bow2
print('tomek' in bow1) # False
print('tomek' in bow3) # True
print('ala' in bow3) # True
print(bow3) # ala:3, ma:3, kota:2, tez:1, tomek:1
    ```
* Odczytywanie częstości wystąpień słów poprzez nawiasy kwadratowe
```
print(bow1["ala"]) # 3
print(bow3["ala"]) # 3
```
* Aktualizację cześtości wystąpień
```
bow3['tomek'] = 10
for el in bow3:
    print el
```
    powinno wyświetlić najpierw `tomek`